**Imports**

In [1]:
import datetime
import glob
import os
import subprocess
from tempfile import TemporaryDirectory

import pandas as pd

**Constants**

In [ ]:
DATETIME_PARSER = '%Y-%m-%dT%H:%MZ'

mass_root = "moose:/adhoc/projects/radar_archive/data/comp/products/composites/"
fname_mass_template = "{dt.year:04d}{dt.month:02d}{dt.day:02d}.tar"
retrieve_path_root = "/scratch/dkillick/precip_rediagnosis/radar/"
variable_fname_template = "{timestamp}_{product}_{resolution}_{area}.gz"

mass_cmd_template = "moo get {args} {src_paths} {dest_path}"
gunzip_cmd_template = "gunzip {zip_file}"
gunzip_cmd_template2 = "gunzip -c {zip_file} > {dest_path}"
# unzip_cmd_template = "unzip {zip_file} -d {dest_root}"  # unzip can't handle files from MASS.
untar_cmd_template = 'tar -xf {path} --directory {dest_root}'

event_start = "2020-02-14T18:00Z"
event_end = "2020-02-17T18:00Z"
date_range = [
    datetime.datetime.strptime(event_start, DATETIME_PARSER),
    datetime.datetime.strptime(event_end, DATETIME_PARSER)
]
archive_time_chunk = 24

**Functions**

In [ ]:
def calc_dates_list(start_datetime, end_datetime, delta_hours):
    dates_to_extract = list(pd.date_range(
        start=start_datetime,
        end=end_datetime - datetime.timedelta(seconds=1),
        freq=datetime.timedelta(
            hours=delta_hours)).to_pydatetime())
    return dates_to_extract

def run_cmd(cmd):
    msg = ""
    try:
        response = subprocess.check_output(cmd, shell=True)
    except subprocess.CalledProcessError as e:
        msg = f"Command errored: {e}"
        raise RetryRequested(max_retries=1, seconds_to_wait=30) from e
    else:
        msg = f"Command returned response: {response}"
    return msg

def retrieve_from_mass(mass_path, dest_path):
    mass_get_cmd = mass_cmd_template.format(
        args="-f",
        src_paths=mass_path,
        dest_path=dest_path
    )
    msg = run_cmd(mass_get_cmd)
    print(msg)

def extract_mass_retrieval(dest_path, mass_tar_name):
    extracted_tar_name = os.path.join(dest_path, mass_tar_name)
    untar_cmd = untar_cmd_template.format(
        path=extracted_tar_name,
        dest_root=dest_path
    )
    msg = run_cmd(untar_cmd)
    print(msg)

def filter_mass_retrieval(dest_path, dest_root):
    # extract_path = os.path.join(dest_path, mass_tar_name)
    filter_products = [
        "nimrod_ng_radar_rainrate_composite",
        "nimrod_ng_radar_qualityproduct_composite",
    ]
    # Get a list of all files to unzip.
    gunzip_files = []
    for product in filter_products:
        variable_fname = variable_fname_template.format(
            timestamp="*",
            product=product,
            resolution="1km",
            area="UK"
        )
        gunzip_files.extend(glob.glob(os.path.join(dest_path, variable_fname)))
    # Unzip each file in turn, moving it to the output directory. 
    for zip_file in gunzip_files:
        filename = os.path.splitext(os.path.basename(zip_file))[0]
        dest_path = os.path.join(dest_root, filename)
        unzip_cmd = gunzip_cmd_template2.format(
            zip_file=zip_file,
            dest_path=dest_path
        )
        msg = run_cmd(unzip_cmd)
        print(f"Extracted {zip_file!r} with message {msg!r}")

def mass_extract_runner(date):
    mass_fname = fname_mass_template.format(dt=date)
    mass_tar_name = os.path.basename(mass_fname)
    mass_path = os.path.join(mass_root, mass_fname)
    retrieve_from_mass(mass_path, retrieve_path_root)
    extract_mass_retrieval(retrieve_path_root, mass_tar_name)
    # filter_mass_retrieval(tempdir, retrieve_path_root)

def mass_extract_runner_v2(date):
    mass_fname = fname_mass_template.format(dt=date)
    mass_tar_name = os.path.basename(mass_fname)
    mass_path = os.path.join(mass_root, mass_fname)
    with TemporaryDirectory(prefix=retrieve_path_root) as tempdir:
        print(f"Temporary extract directory: {tempdir}")
        retrieve_from_mass(mass_path, tempdir)
        extract_mass_retrieval(tempdir, mass_tar_name)
        filter_mass_retrieval(tempdir, retrieve_path_root)

In [ ]:
dates_to_extract = calc_dates_list(
    datetime.datetime(date_range[0].year,
                      date_range[0].month,
                      date_range[0].day, 0, 0),
    datetime.datetime(date_range[1].year,
                      date_range[1].month,
                      date_range[1].day, 23, 59),
    archive_time_chunk
)
dates_to_extract

In [ ]:
mass_extract_runner_v2(dates_to_extract[0])

In [ ]:
mass_paths = [os.path.join(mass_root, fname_mass_template.format(dt=dt)) for dt in dates_to_extract]
mass_paths

In [ ]:
mass_radar_get_cmd = mass_cmd_template.format(
    src_paths=mass_paths[0],
    dest_path=save_path,
    args='-f',
)
mass_radar_get_cmd

In [ ]:
op = subprocess.check_output(mass_radar_get_cmd, shell=True)
op

In [ ]:
import os
import shutil
from tempfile import TemporaryDirectory, mkdtemp 

class EncapsulatedTemporaryDir(object):
    def __init__(self, prefix, suffix=None):
        self.prefix = prefix
        if not self.prefix.endswith(os.sep):
            self.prefix += os.sep
        self.suffix = suffix
        if self.suffix is not None and not self.suffix.startswith(os.sep):
            self.suffix = os.sep + self.suffix

        self._tempdir = None
        # self.open()
        self.mk_tempdir()

    @property
    def tempdir(self):
        if self._tempdir is None:
            self.mk_tempdir()
        return self._tempdir

    @tempdir.setter
    def tempdir(self, value):
        self._tempdir = value

    @property
    def name(self):
        # return self.tempdir.name
        return self.tempdir

    def open(self):
        self.tempdir = TemporaryDirectory(
            prefix=self.prefix,
            suffix=self.suffix
        )

    def close(self):
        self.tempdir.cleanup()

    def mk_tempdir(self):
        self.tempdir = mkdtemp(prefix=self.prefix, suffix=self.suffix)

    def rm_tempdir(self):
        shutil.rmtree(self.name)

In [ ]:
tempdir = EncapsulatedTemporaryDir("/scratch/dkillick/precip_rediagnosis/radar/")
print(tempdir.name)

In [ ]:
tempdir.rm_tempdir()

In [ ]:
products = ["nimrod_ng_radar_rainrate_composite", "nimrod_ng_radar_qualityproduct_composite"]
temp_dir = os.path.join("/scratch/dkillick/precip_rediagnosis/radar", "5o3p5ap_")

gunzip_files = []
for product in products:
    variable_fname = variable_fname_template.format(
        timestamp="*",
        product=product,
        resolution="1km",
        area="UK"
    )
    print(variable_fname)
    gunzip_files.extend(glob.glob(os.path.join(temp_dir, variable_fname)))
    
print(len(gunzip_files))

In [4]:
import hashlib

In [8]:
hashlib.blake2b(salt=os.urandom(hashlib.blake2b.SALT_SIZE)).hexdigest()[:16]

'47213015470f349b'

In [8]:
hashlib.algorithms_available

{'blake2b',
 'blake2s',
 'md5',
 'sha1',
 'sha224',
 'sha256',
 'sha384',
 'sha3_224',
 'sha3_256',
 'sha3_384',
 'sha3_512',
 'sha512',
 'shake_128',
 'shake_256'}

In [2]:
os.urandom?

Signature: os.urandom(size, /)
Docstring: Return a bytes object containing random bytes suitable for cryptographic use.
Type:      builtin_function_or_method
